In [5]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
import matplotlib.pyplot as plt
from time import time
from sklearn.model_selection import train_test_split

%matplotlib inline

In [4]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 25s 0us/step


### 数据预处理

In [ ]:
# 以channels来划分，计算均值、标准差
np.mean(x_train, axis=(0,1,2)) # == x_mean
np.std(x_train, axis=(0,1,2)) # == x_std

In [7]:
def color_preprocessing(x_train, x_test):
    # every channel : mean = 0, std = 1
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_mean = [125.307,  122.950,  113.865]
    x_std = [62.993,  62.089,  66.705]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - x_mean[i]) / x_std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - x_mean[i]) / x_std[i]
    return x_train, x_test

In [8]:
# mean = 0, std = 1
x_train, x_test = color_preprocessing(x_train, x_test)

# one_hot
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [9]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

In [10]:
x_train[0,:5,:5,0]

array([[-1.052609  , -1.3066055 , -1.195482  , -0.909736  , -0.4334926 ],
       [-1.7352245 , -1.989221  , -1.703475  , -1.1796073 , -0.5922404 ],
       [-1.5923516 , -1.7352245 , -1.2113568 , -0.6716143 , -0.24299525],
       [-1.4653533 , -1.3859794 , -0.6081152 , -0.30649436, -0.16362134],
       [-1.195482  , -1.052609  , -0.36999348,  0.02687602, -0.02074832]],
      dtype=float32)

In [11]:
train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

train_x.shape, valid_x.shape, train_y.shape, valid_y.shape

((45000, 32, 32, 3), (5000, 32, 32, 3), (45000, 10), (5000, 10))

In [12]:
weight_decay = 0.0001 # L2 

In [13]:
def build_model():
    # from github : https://github.com/BIGBALLON/cifar-10-cnn/blob/master/2_Network_in_Network/Network_in_Network_keras.py
    # strange cnovnet
    model = Sequential()
    
    model.add(Conv2D(192, (5,5), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay), input_shape=(32,32,3))) 
    model.add(Conv2D(160, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) 
    model.add(Conv2D(96, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(192, (5,5), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(192, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(192, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) 
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(192, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(192, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(10, (1,1), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) 
    
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [14]:
model = build_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 192)       14592     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 160)       30880     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 96)        15456     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 192)       460992    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 192)       37056     
__________

In [15]:
# data augmentation
datagen = ImageDataGenerator(horizontal_flip=True, 
                             width_shift_range=0.125, 
                             height_shift_range=0.125, 
                             fill_mode='constant', cval=0.)

In [16]:
datagen.fit(train_x)

In [19]:
history = model.fit_generator(datagen.flow(train_x, train_y, batch_size=32), steps_per_epoch=1407, epochs=10, 
                              validation_data=datagen.flow(valid_x, valid_y, batch_size=32), 
                              validation_steps=157)
model.save('cnn_cifar10.h5')

Epoch 1/10
1407/1407 [==============================] - 62s 44ms/step - loss: 2.0022 - acc: 0.3308 - val_loss: 1.9232 - val_acc: 0.3620
Epoch 2/10
1407/1407 [==============================] - 62s 44ms/step - loss: 1.9088 - acc: 0.3763 - val_loss: 1.8363 - val_acc: 0.4042
Epoch 3/10
1407/1407 [==============================] - 62s 44ms/step - loss: 1.8479 - acc: 0.4019 - val_loss: 1.7895 - val_acc: 0.4270
Epoch 4/10
1407/1407 [==============================] - 62s 44ms/step - loss: 1.8010 - acc: 0.4229 - val_loss: 1.7144 - val_acc: 0.4500
Epoch 5/10
1407/1407 [==============================] - 62s 44ms/step - loss: 1.7681 - acc: 0.4375 - val_loss: 1.6916 - val_acc: 0.4630
Epoch 6/10
1407/1407 [==============================] - 62s 44ms/step - loss: 1.7322 - acc: 0.4503 - val_loss: 1.6776 - val_acc: 0.4668
Epoch 7/10
1407/1407 [==============================] - 62s 44ms/step - loss: 1.7109 - acc: 0.4585 - val_loss: 1.6473 - val_acc: 0.4894
Epoch 8/10
1407/1407 [==========================

In [20]:
# validation without data augmentation

model = build_model()
val_datagen = ImageDataGenerator()
val_datagen.fit(valid_x)
history = model.fit_generator(datagen.flow(train_x, train_y, batch_size=32), steps_per_epoch=1407, epochs=5, 
                              validation_data=val_datagen.flow(valid_x, valid_y, batch_size=32), 
                              validation_steps=157)
model.save('cnn1_cifar10.h5')

Epoch 1/5
1407/1407 [==============================] - 62s 44ms/step - loss: 2.1951 - acc: 0.1925 - val_loss: 2.0852 - val_acc: 0.2562
Epoch 2/5
1407/1407 [==============================] - 62s 44ms/step - loss: 2.0170 - acc: 0.2985 - val_loss: 1.9921 - val_acc: 0.2914
Epoch 3/5
1407/1407 [==============================] - 62s 44ms/step - loss: 1.9223 - acc: 0.3406 - val_loss: 1.8630 - val_acc: 0.3584
Epoch 4/5
1407/1407 [==============================] - 62s 44ms/step - loss: 1.8559 - acc: 0.3704 - val_loss: 1.8157 - val_acc: 0.3864
Epoch 5/5
1407/1407 [==============================] - 62s 44ms/step - loss: 1.8071 - acc: 0.3920 - val_loss: 1.7215 - val_acc: 0.4132


In [21]:
def build_model_vgg16():
    # VGG16 without BN
    model = Sequential()
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay), input_shape=(32,32,3))) 
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 32x32x64
    model.add(MaxPooling2D(pool_size=(2,2))) # 16x16x64
    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 16x16x128
    model.add(MaxPooling2D(pool_size=(2,2))) # 8x8x128
    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 8x8x256
    model.add(MaxPooling2D(pool_size=(2,2))) # 4x4x256
    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 4x4x512
    model.add(MaxPooling2D(pool_size=(2,2))) # 2x2x512
    
    model.add(Dropout(0.75))
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 2x2x512
    model.add(MaxPooling2D(pool_size=(2,2))) # 1x1x512
    
    model.add(GlobalAveragePooling2D()) # 512
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [22]:
model_vgg = build_model_vgg16()
print(model_vgg.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 128)         0         
__________

In [23]:
# failed to train

history = model_vgg.fit_generator(datagen.flow(train_x, train_y, batch_size=32), steps_per_epoch=1500, epochs=5, 
                                  validation_data=datagen.flow(valid_x, valid_y, batch_size=32), 
                                  validation_steps=157)
model.save('vgg_cifar10.h5')

Epoch 1/5
1500/1500 [==============================] - 142s 94ms/step - loss: 2.3193 - acc: 0.0993 - val_loss: 2.3026 - val_acc: 0.1006
Epoch 2/5
1500/1500 [==============================] - 139s 92ms/step - loss: 2.3028 - acc: 0.0966 - val_loss: 2.3027 - val_acc: 0.0974
Epoch 3/5
1500/1500 [==============================] - 138s 92ms/step - loss: 2.3028 - acc: 0.0990 - val_loss: 2.3027 - val_acc: 0.1006
Epoch 4/5
1500/1500 [==============================] - 138s 92ms/step - loss: 2.3028 - acc: 0.0991 - val_loss: 2.3027 - val_acc: 0.0952
Epoch 5/5
1500/1500 [==============================] - 138s 92ms/step - loss: 2.3028 - acc: 0.0959 - val_loss: 2.3027 - val_acc: 0.1008


In [24]:
from keras.layers import BatchNormalization, Flatten

def build_model_vgg16_1():
    # VGG16 with BN and more Dropout -- Better
    model = Sequential()
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay), input_shape=(32,32,3))) 
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 32x32x64
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2))) # 16x16x64
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 16x16x128
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2))) # 8x8x128
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 8x8x256
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2))) # 4x4x256
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 4x4x512
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2))) # 2x2x512
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay))) # 2x2x512
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2))) # 1x1x512
    
    model.add(Flatten()) # 512
    model.add(Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [25]:
model_vgg1 = build_model_vgg16_1()
print(model_vgg1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 16, 16, 128)       73856     
__________

In [26]:
history = model_vgg1.fit_generator(datagen.flow(train_x, train_y, batch_size=32), steps_per_epoch=1500, epochs=5, 
                                   validation_data=datagen.flow(valid_x, valid_y, batch_size=32), 
                                   validation_steps=157)
model.save('vgg_cifar10_1.h5')

Epoch 1/5
1500/1500 [==============================] - 192s 128ms/step - loss: 2.4336 - acc: 0.3032 - val_loss: 2.1793 - val_acc: 0.3712
Epoch 2/5
1500/1500 [==============================] - 188s 125ms/step - loss: 1.9284 - acc: 0.4567 - val_loss: 1.7968 - val_acc: 0.5280
Epoch 3/5
1500/1500 [==============================] - 188s 125ms/step - loss: 1.7012 - acc: 0.5488 - val_loss: 1.6032 - val_acc: 0.5916
Epoch 4/5
1500/1500 [==============================] - 188s 125ms/step - loss: 1.5463 - acc: 0.6094 - val_loss: 1.7229 - val_acc: 0.5832
Epoch 5/5
1500/1500 [==============================] - 188s 125ms/step - loss: 1.5171 - acc: 0.6473 - val_loss: 1.5653 - val_acc: 0.6302
